In [1]:
# import libraries here

import requests
import pandas as pd
import json

# your code here
PUNKIPA = 'https://api.punkapi.com/v2/beers'

response = requests.get(PUNKIPA)
beer = response.json()


In [2]:
beer_df = pd.DataFrame(beer)
newbeer = beer_df[['name','abv','tagline','srm','ebc','food_pairing','ingredients','ph','method']]

newbeer.head()

,name,abv,tagline,srm,ebc,food_pairing,ingredients,ph,method
0,Buzz,4.5,A Real Bitter Experience.,10.0,20.0,"[Spicy chicken tikka masala, Grilled chicken q...","{'malt': [{'name': 'Maris Otter Extra Pale', '...",4.4,"{'mash_temp': [{'temp': {'value': 64, 'unit': ..."
1,Trashy Blonde,4.1,You Know You Shouldn't,15.0,15.0,"[Fresh crab with lemon, Garlic butter dipping ...","{'malt': [{'name': 'Maris Otter Extra Pale', '...",4.4,"{'mash_temp': [{'temp': {'value': 69, 'unit': ..."
2,Berliner Weisse With Yuzu - B-Sides,4.2,Japanese Citrus Berliner Weisse.,4.0,8.0,"[Smoked chicken wings, Miso ramen, Yuzu cheese...","{'malt': [{'name': 'Propino Pale Malt', 'amoun...",3.2,"{'mash_temp': [{'temp': {'value': 60, 'unit': ..."
3,Pilsen Lager,6.3,Unleash the Yeast Series.,15.0,30.0,"[Spicy crab cakes, Spicy cucumber and carrot T...","{'malt': [{'name': 'Extra Pale', 'amount': {'v...",4.4,"{'mash_temp': [{'temp': {'value': 65, 'unit': ..."
4,Avery Brown Dredge,7.2,Bloggers' Imperial Pilsner.,5.0,10.0,"[Vietnamese squid salad, Chargrilled corn on t...","{'malt': [{'name': 'Lager Malt', 'amount': {'v...",4.4,"{'mash_temp': [{'temp': {'value': 66, 'unit': ..."


In [3]:
def get_dur(row):
    return row['method']['mash_temp'][0]['duration']
def get_temp_val(row):
    return row['method']['mash_temp'][0]['temp']['value']

Apply function to pass as arguments to the functions and be stored in a variable

In [4]:
mash_duration = newbeer.apply(get_dur, axis=1)
mash_temp = newbeer.apply(get_temp_val, axis=1)

Create two new columns based on the two variables stored above

In [5]:
newbeer['mash_duration'] = mash_duration.copy()
newbeer['mash_temp'] = mash_temp.copy()
newbeer = newbeer.drop(['food_pairing', 'ingredients', 'method'], axis=1)
newbeer.head()

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,name,abv,tagline,srm,ebc,ph,mash_duration,mash_temp
0,Buzz,4.5,A Real Bitter Experience.,10.0,20.0,4.4,75.0,64
1,Trashy Blonde,4.1,You Know You Shouldn't,15.0,15.0,4.4,NaN,69
2,Berliner Weisse With Yuzu - B-Sides,4.2,Japanese Citrus Berliner Weisse.,4.0,8.0,3.2,10.0,60
3,Pilsen Lager,6.3,Unleash the Yeast Series.,15.0,30.0,4.4,NaN,65
4,Avery Brown Dredge,7.2,Bloggers' Imperial Pilsner.,5.0,10.0,4.4,70.0,66


Create another column grouping srm values in three bins

In [6]:
#pd.cut(newbeer.srm, bins=3, labels=['Low', 'Med', 'High'])

newbeer['srm_type'] = pd.cut(newbeer.srm, 
                            bins=3, 
                            labels=['Low', 'Med', 'High'], 
                            right=True, 
                            include_lowest=True)

In [7]:
newbeer.sort_values(by='srm_type')

,name,abv,tagline,srm,ebc,ph,mash_duration,mash_temp,srm_type
0,Buzz,4.50,A Real Bitter Experience.,10.0,20.0,4.4,75.0,64,Low
21,Devine Rebel (w/ Mikkeller),12.50,Oak-aged Barley Wine.,18.0,36.0,4.4,90.0,65,Low
20,Vice Bier,4.30,Hoppy Wheat Bier.,15.0,30.0,4.0,75.0,65,Low
19,Rabiator,10.27,Imperial Wheat Beer,12.0,24.0,4.4,60.0,64,Low
17,Russian Doll – India Pale Ale,6.00,Nesting Hop Bomb.,12.5,25.0,5.2,75.0,65,Low
16,AB:06,11.20,Imperial Black IPA.,35.0,70.0,NaN,90.0,65,Low
14,Mixtape 8,14.50,"An Epic Fusion Of Old Belgian, American New Wa...",20.0,40.0,4.4,40.0,65,Low
13,Alpha Dog,4.50,Existential Red Ale.,31.0,62.0,4.4,25.0,65,Low
22,Storm,8.00,Islay Whisky Aged IPA.,6.0,12.0,4.4,75.0,65,Low
24,Bad Pixie,4.70,Spiced Wheat Beer.,4.0,8.0,4.4,75.0,67,Low


In [8]:
#Group by srm new col.
#Check whether there is a relationship with duration or temp or not AND ph.
newbeer.groupby(['srm_type']).mean()

,abv,srm,ebc,ph,mash_duration,mash_temp
srm_type,,,,,,
Low,7.039412,13.352941,25.823529,4.350,65.666667,65.00
Med,8.875000,53.000000,106.000000,5.125,83.750000,67.25
High,5.700000,104.750000,209.500000,4.300,35.000000,66.50


In [9]:
newbeer['ingredients'][1]

KeyError: 'ingredients'

In [ ]:
import pandas as pd
newbeer.corr()

In [ ]:
newbeer.corr().loc['mash_duration']